In [ ]:
import random
import re
import time

import openai
openai.api_key = 'FIX ME: insert key here'

current_tokens = 0
# Maximum tokens program is allowed to request before timing out
max_tokens = 400
def generate_words_and_spend_money(prompt, poem):
    global current_tokens
    global max_tokens
    if current_tokens >= max_tokens:
        print('Tokens expended')
        print(poem)
        raise TimeoutError
    current_tokens += 1
    line = openai.Completion.create(
    model="text-davinci-003",
    prompt="{}:\n\n{}".format(prompt, poem),
    max_tokens=1,
    temperature=0.9,
    logprobs=5
    )
    choices = line["choices"][0]["logprobs"]["top_logprobs"][0]
    words = []
    record = set()
    for choice in choices.keys():
        word = re.sub('(^ +)|( +$)', '', choice)
        id = re.sub('[^A-Z]', '', word.upper())
        prob = choices[choice]
        if len(poem) > 0 and poem[-1] == '\n' and choice.find('\n') != -1:
            continue
        if id not in record and choice.find('\\') == -1 and choice.find('/') == -1 and prob > -10 and re.sub('[0-9]', '', choice) != '':
            done = False
            for i in range(len(words)):
                if choices[words[i]] < prob:
                    words.insert(i, choice)
                    done = True
                    break
            if not done:
                words.append(choice)
            record.add(id)
    print(words)
    print(choices)
    time.sleep(3)
    return words

# Loads the modified CMU dictionary and the list of unstressed words
def load_data():
    cmu_dictionary = {}
    unstressed_words = set()
    with open('Modified_CMU.txt','r',encoding="utf8") as f:
        for line in f:
            entry = line.replace("\n", "").split('  ')
            if entry[0] in cmu_dictionary:
                cmu_dictionary[entry[0]].append(entry[1])
            else:
                cmu_dictionary[entry[0]] = [entry[1]]
    cmu_dictionary[''] = ['']
    with open('unstressed_words.txt','r',encoding="utf8") as f:
        for line in f:
            unstressed_words.add(line.replace("\n", ""))
    return cmu_dictionary, unstressed_words

# Given a list of words representing a line, returns a list of sets of all possible stress patterns for each word in the line in order
def get_line_stress_patterns(line, cmu_dictionary, unstressed_words):
    stress_patterns = []
    for word in line:
        stress_patterns.append(get_stress_patterns(word, cmu_dictionary, unstressed_words))
    return stress_patterns

# Returns a set of the possible stress patterns of the input word
def get_stress_patterns(word, cmu_dictionary, unstressed_words):
    word = word.upper()
    if word in unstressed_words:
        return {'\''}
    if word not in cmu_dictionary:
        return set()
    stress_patterns = set()
    for entry in cmu_dictionary[word]:
        stress_patterns.add(cmu_to_stress_pattern(entry))
    return stress_patterns

# Given an entry from the CMU dictionary, converts it to a stress pattern
def cmu_to_stress_pattern(entry):
    stress_as_numbers = re.sub('[^0-2]', '', entry)
    if len(stress_as_numbers) == 1:
        return stress_as_numbers.replace('0', '.').replace('2', '~').replace('1', '~')
    if stress_as_numbers.find('1') > -1:
        return stress_as_numbers.replace('0', '.').replace('2', '.').replace('1', '-')
    else:
        return stress_as_numbers.replace('0', '.').replace('2', '-')

# Checks if the given line matches any of the possible meter stresses
# 0 means no, 1 means on track, 2 means yes
def matches_meter(line, meter_stresses, cmu_dictionary, unstressed_words):
    return some_stress_pattern_matches('', get_line_stress_patterns(line, cmu_dictionary, unstressed_words), meter_stresses)

# Checks if a given representation of possible line stresses can match a given representation of meter stresses
# 0 means no, 1 means on track, 2 means yes
def some_stress_pattern_matches(line_stress, stress_patterns, meter_stresses):
    match_type = 0
    if len(stress_patterns) == 0:
        for meter_stress in meter_stresses:
            result = stresses_match(line_stress, meter_stress)
            if result > match_type:
                match_type = result
                if match_type == 2:
                    return match_type
        return match_type
    for stress_pattern in stress_patterns[0]:
        result = some_stress_pattern_matches(line_stress + stress_pattern, stress_patterns[1:], meter_stresses)
        if result > match_type:
            match_type = result
            if match_type == 2:
                return match_type
    return match_type

# Return a tuple of match (0 = no, 1 = partial, 2 = full), pattern (list of stress patterns
# corresponding to each word in the line)
def scan_line(line, meter_stresses, cmu_dictionary, unstressed_words):
    stress_patterns = get_line_stress_patterns(line, cmu_dictionary, unstressed_words)
    return find_matching_stress_pattern([], stress_patterns, meter_stresses)

# Return a tuple of match, pattern, in which match is 0 no match, 1 part match, 2 full match,
# and pattern is a list of the stresses in the order used
def find_matching_stress_pattern(line_stress, stress_patterns, meter_stresses):
    match_type = 0
    match_stress = []
    if len(stress_patterns) == 0:
        for meter_stress in meter_stresses:
            result = stress_list_matches(line_stress, meter_stress)
            if result > match_type:
                match_type = result
                match_stress = line_stress
                if match_type == 2:
                    return match_type, match_stress
        return match_type, match_stress
    for stress_pattern in stress_patterns[0]:
        line_stress.append(stress_pattern)
        result = find_matching_stress_pattern(line_stress, stress_patterns[1:], meter_stresses)
        if result[0] > match_type:
            match_type = result[0]
            match_stress = result[1]
            if match_type == 2:
                return match_type, match_stress
        line_stress.pop()
    return match_type, match_stress

# Checks if the pattern in line_stress matches with the pattern in meter_stress
# 0 means no, 1 means on track, 2 means yes
def stresses_match(line_stress, meter_stress):
    if len(line_stress) > len(meter_stress):
        return 0
    match_type = 2
    while len(line_stress) < len(meter_stress):
        line_stress += '.'
        match_type = 1
    padded_line_stress = '.' + line_stress + '.'
    for x in range(len(meter_stress)):
        if line_stress[x] == '-' and meter_stress[x] == '.':
            return 0
        if meter_stress[x] == '-' and re.match('.\'[~-] | [~-]\'.', padded_line_stress[x : x + 3]) is not None:
            return 0
        if meter_stress[x] == '.' and re.match('[.\']~[.\']', padded_line_stress[x : x + 3]) is not None:
            return 0
    return match_type

# Generates a random list of words from the dictionary
def generate_words(cmu_dictionary):
    words = []
    for x in range(random.randrange(3, 9)):
        words.append(random.choice(list(cmu_dictionary)))
    return words

# Generates a list representing a line using generate_words in the given meter
def generate_line(line, meter_stresses, cmu_dictionary, unstressed_words):
    match = matches_meter(line, meter_stresses, cmu_dictionary, unstressed_words)
    if match == 2:
        return line
    if match == 1:
        for word in generate_words(cmu_dictionary):
            line.append(word)
            result = generate_line(line, meter_stresses, cmu_dictionary, unstressed_words)
            if result is not None:
                return result
            line.pop()
        return None
    return None

# Given a word, returns a set of all possible rhyme patterns represented as strings
def get_rhyme_patterns(word, cmu_dictionary):
    word = word.upper()
    if word not in cmu_dictionary:
        return set()
    rhyme_patterns = set()
    for entry in cmu_dictionary[word]:
        rhyme_patterns.update(cmu_to_rhyme_patterns(entry))
    return rhyme_patterns

# Given an entry from the CMU dictionary, converts it to a list of possible rhyme patterns represented as strings
def cmu_to_rhyme_patterns(entry):
    match = re.search('([A-Z]+1)( [A-Z02]+)*$', entry)
    if match is not None:
        return [match.group(0)]
    match = re.search('([A-Z]+2)( [A-Z0]+)*$', entry)
    if match is not None:
        return [match.group(0)]
    phonemes = entry.split(' ')
    index = len(phonemes) - 1
    rhyme_patterns = ['']
    while index >= 0:
        rhyme_patterns[-1] = phonemes[index] + ' ' + rhyme_patterns[-1]
        if phonemes[index].find('0') != -1:
            rhyme_patterns.insert(-1, rhyme_patterns[-1][:-1])
        index -= 1
    return rhyme_patterns[:-1]

# Given two words, returns True if the words rhyme and False if they do not
def words_rhyme(word1, word2, cmu_dictionary):
    rhymes1 = get_rhyme_patterns(word1, cmu_dictionary)
    rhymes2 = get_rhyme_patterns(word2, cmu_dictionary)
    for rhyme1 in rhymes1:
        for rhyme2 in rhymes2:
            if rhyme1 == rhyme2:
                return True
    return False

# Given two lines represented by lists of words, return True if the lines rhyme and False if they do not
def lines_rhyme(line1, line2, cmu_dictionary):
    return words_rhyme(line1[-1], line2[-1], cmu_dictionary)

# Concatenate a list of tokens
def tokens_to_text(tokens):
    text = ''
    for token in tokens:
        text += token
    return text

# Convert a string into a list representing a line
def line_text_to_list(line):
    return re.sub('(^ )|( $)', '', re.sub(' +', ' ', re.sub('[^A-Z ]', ' ', line.upper()))).split(' ')

# Convert text into a list of lists representing lines
def text_to_line_list(text):
    lines = []
    for line in re.sub('(^\n)|(\n$)', '', re.sub('\n+', '\n', re.sub('[^A-Z \n]', '', text.upper()))).split('\n'):
        line_list = line_text_to_list(line)
        if len(line_list) != 1 or line_list[0] != '':
            lines.append(line_list)
    if len(lines) == 0:
        return ['']
    return lines

# Return 2 if the poem is an exact match, 1 if it is a partial match, and 0 if it does not match
def poem_matches(text, meter_stresses_list, rhyme_scheme, cmu_dictionary, unstressed_words):

    lines = text_to_line_list(text)

    if len(lines) > len(rhyme_scheme):
        return 0

    last_match = matches_meter(lines[-1], meter_stresses_list[len(lines) - 1], cmu_dictionary, unstressed_words)

    match_type = 2
    
    if len(text) > 0 and text[-1] != '!' and  text[-1] != '.' and text[-1] != '>' and text[-1] != ',':
        match_type = 1

    if last_match == 0:
        return 0
    
    if last_match == 1:
        lines.pop()

    if len(lines) != len(rhyme_scheme):
        match_type = 1
    
    rhyming_lines = {}

    for i in range(len(lines)):
        if rhyme_scheme[i] != '*' and rhyme_scheme[i] in rhyming_lines:
            rhyming_lines[rhyme_scheme[i]].append(i)
        else:
            rhyming_lines[rhyme_scheme[i]] = [i]
    
    for i in range(len(lines)):
        match = matches_meter(lines[i], meter_stresses_list[i], cmu_dictionary, unstressed_words)
        if match != 2:
            return 0
        for j in rhyming_lines[rhyme_scheme[i]]:
            if j >= i:
                break
            if not lines_rhyme(lines[i], lines[j], cmu_dictionary):
                return 0
    
    return match_type


# Generates a random list of words from the dictionary
def generate_words_and_lines(cmu_dictionary):
    words = random.choice([[], ['\n']])
    for x in range(random.randrange(3, 9)):
        words.append(random.choice(list(cmu_dictionary)))
    return words

# Generates a list representing a line using generate_words in the given meter
def generate_poem(text, meter_stresses, rhyme_scheme, cmu_dictionary, unstressed_words):
    match = poem_matches(text, meter_stresses, rhyme_scheme, cmu_dictionary, unstressed_words)
    if match == 2:
        return text
    if match == 1:
        for word in generate_words_and_lines(cmu_dictionary):
            text += ' ' + word
            result = generate_poem(text, meter_stresses, rhyme_scheme, cmu_dictionary, unstressed_words)
            if result is not None:
                return result
            text = re.sub(' [^ ]+$', '', text)
        return None
    return None

# Generates a list representing a line using generate_words in the given meter
def generate_poem_and_spend_money(prompt, text, meter_stresses, rhyme_scheme, cmu_dictionary, unstressed_words):
    match = poem_matches(text, meter_stresses, rhyme_scheme, cmu_dictionary, unstressed_words)
    if match == 2:
        return text
    if match == 1:
        for word in generate_words_and_spend_money(prompt, text):
            text += word
            print(text)
            result = generate_poem_and_spend_money(prompt, text, meter_stresses, rhyme_scheme, cmu_dictionary, unstressed_words)
            if result is not None:
                return result
            text = text[:len(text) - len(word)]
        return None
    return None

# Load necessary files
cmu_dictionary, unstressed_words = load_data()

sonnet_stresses = ['.-.-.-.-.-', '-..-.-.-.-', '.-.-.-.-.-.', '-..-.-.-.-.']
five = ['*****']
seven = ['*******']

sonnet_lines = []
for i in range(14):
    sonnet_lines.append(sonnet_stresses)

haiku_lines = [five, seven, five]

sonnet_rhyme_scheme = 'ABABCDCDEFEFGG'
haiku_rhyme_scheme = '***'

haiku = generate_poem_and_spend_money('Generate a haiku about linguistics:', '', haiku_lines, haiku_rhyme_scheme, cmu_dictionary, unstressed_words)
print(current_tokens)
print(max_tokens)

sonnet = generate_poem_and_spend_money('Generate a sonnet about machine learning:', '', sonnet_lines, sonnet_rhyme_scheme, cmu_dictionary, unstressed_words)
print(current_tokens)
print(max_tokens)

print(haiku)
print(sonnet)

#print(scan_line(line_text_to_list('And help us to make a better land!'), sonnet_stresses, cmu_dictionary, unstressed_words))

In [ ]:
!pip install openai
!pip install spacy_syllables
import os
import openai
import spacy
from spacy_syllables import SpacySyllables
openai.api_key = "API_KEY_HERE"
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 16.3 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=94a5a5ce4560bf03c1b8fdd0aef6b99a2e153501f749bbffb743e1fe8e2e686b
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 6.2 MB/s 


[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
!sudo apt-get install libenchant1c2a
!pip install pyenchant
#!pip3 install pyenchant==1.6.6
import enchant
d = enchant.Dict("en_US")
d.check("\n")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 7 not upgraded.
Need to get 1,312 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.

False

In [ ]:
def get_syllable_word(word):
  """
  input a string word
  uses cmu_dictionary to get stress digits of word
  output is total syllables of the word
  """

  stress = cmudict.dict()[word.lower()] # using stress to count syllables 
  syllable_count=0

  for string in stress[0]:
    for x in string:
      if any(char.isdigit() for char in x): #count digits
        syllable_count += 1

  return syllable_count

def generate_line(doc, count):
  """
  generates line form poem. 
  Input gpt-3 doc, and count for line
  takes care of cases with punctuations and newline char from doc,
  puts spaces after words, and removes additional spaces
  outputs the line and the total syllables in line
  """
  curr_text = ""
  sent_count = 0 
  for word in doc:
    word_count = 0
    if word.isalpha():
      word_count = get_syllable_word(word)
    
      if (sent_count + word_count) > count:
        return curr_text[:-1], sent_count
      else:
        curr_text += word + " "
        sent_count += word_count
    
    else:
      if (word == "\n" or word == "\n\n") and (sent_count == 0):
          curr_text = curr_text[:-1]
          curr_text += word
      else:
          return curr_text[:-1], sent_count
       
  return curr_text[:-1], sent_count

def gen_text(prompt, num_tokens, poem):
  tokens_gen = num_tokens

  # create first line
  #when poem is empty
  if poem == '':
    line = openai.Completion.create(
    model="text-davinci-003",
    prompt="Generate a haiku about with the prompt: {}".format(prompt),
    max_tokens=tokens_gen,
    temperature=0,
    logprobs=5
    )
    return line
# when the poem is not empty
  line = openai.Completion.create(
  model="text-davinci-003",
  prompt="Write the next line of a haiku that starts with the line(s): \"{}\". The prompt of the poem is: {}".format(poem,prompt),
  max_tokens=tokens_gen,
  temperature=0,
  logprobs=5
  )
  return line

def choose_new_token(line, curr_text, sent_count, line_count):
  english = enchant.Dict("en_US")
  log_text = ""
  #print(line)
  #iterations = []
  #print(line["choices"][0]["logprobs"]["top_logprobs"])
  for option in line["choices"][0]["logprobs"]["top_logprobs"]:
    if log_text != curr_text:
      log_text += max(option, key=option.get)
      #iterations.append([log_text, curr_text, log_text==curr_text])
    else:
      print("EQUAL!")
      selected = max(option, key=option.get)
      option.pop(selected)
      while len(option) > 0:
        word_count = 0
        selected = max(option, key=option.get)
        if selected == " ":
          log_text += selected
          #print(iterations)
          return log_text, sent_count
        if selected == ".": # edge case where . passes through the english word check
          option.pop(selected)
          continue
        #print(selected, word_count)
        if english.check(selected.replace(" ", "")):
          word_count = get_syllable_word(selected.replace(" ", ""))
        
          if (sent_count + word_count) == line_count:
            #print(selected, word_count, sent_count, log_text)
            log_text += selected
            sent_count += word_count
            #print(iterations)
            return log_text, sent_count
        option.pop(selected)
  #print(iterations)           
  return log_text, sent_count

def find_last_word(sent, num_syllables):
  english = enchant.Dict("en_US")
  last_word = openai.Completion.create(
  model="text-davinci-003",
  prompt="Finish this sentence of a poem with one word that best fits.: \"{}\"".format(sent),
  max_tokens=5,
  temperature=0,
  logprobs=5
  )
  ans = last_word["choices"][0]["text"].replace("\n", "")

  if english.check(ans) and get_syllable_word(ans) == num_syllables:
    return ans
  else:
    word = ""
    token_count = 0
    for option in last_word["choices"][0]["logprobs"]["top_logprobs"]:
      if max(option, key=option.get).isalpha():
        word += max(option, key=option.get)
        if english.check(word):
          if get_syllable_word(word) == num_syllables:
            return word
          else:
            break
      token_count += 1
    
    while token_count > 0:
      for i in last_word["choices"][0]["logprobs"]["top_logprobs"][token_count]:
        if i.isalpha():
          if english.check(i):
            if get_syllable_word(i) == num_syllables:
              return i
      token_count -= 1
    
    return "ERROR: NO GOOD OPTIONS"

def generate_poem(prompt, num_syllables, num_tokens, poem):
  nlp = spacy.load("en_core_web_sm")
  nlp.add_pipe("syllables", after="tagger", config={"lang": "en_US"})
  line = gen_text(prompt, num_tokens, poem)
  #print(line["choices"][0]["text"])
  text = line["choices"][0]["text"]#[len(poem):]
  doc = nlp(text)
  doc = [(token.text) for token in doc]
  #counter = 0
  #if len(poem) != 0:
  #  substring = ""
  #  for i in doc:
  #    substring += i
  #    if poem.find(substring) == -1:
  #      break
  #    counter += 1
  #doc = doc[counter:]
  #print("gen poem", doc)
  #print(poem)
  curr_text = generate_line(doc, num_syllables)
  print(curr_text)
  if curr_text[1] != num_syllables:
    new_text = choose_new_token(line, curr_text[0], curr_text[1], num_syllables)
    print(new_text)
    if new_text[1] != num_syllables:
      print(num_syllables - new_text[1])
      last = find_last_word(new_text[0], num_syllables - new_text[1])
      return poem + new_text[0] + last
    return poem + new_text[0]
  
  return poem + curr_text[0]

def haiku_v3(prompt):
  
  line1 = generate_poem(prompt, 5, 15, "")
  #print("print line 1", line1)
  line2 = generate_poem(prompt, 7, 45, line1)
  #print("print line 2", line2)
  line3 = generate_poem(prompt, 5, 70, line2)
  #print("print line 3", line3)
  return line3 

In [ ]:
haiku_v3("")

In [ ]:
haiku_v3("Astrology girl.")

('\n\nMysterious girl', 5)
('\n\nHer eyes tell stories of the', 7)
('\n\nPast lives and stars', 4)
EQUAL!
('\n\nPast lives and stars that', 5)


'\n\nMysterious girl\n\nHer eyes tell stories of the\n\nPast lives and stars that'

In [ ]:
%%time
haiku_v3("Opening a door to the mind when using drugs")

('\n\nOpening a door', 5)
('\n\nA new world awaits', 5)
EQUAL!
('\n\nA new world awaits inside', 7)
('\n\nThe mind expands', 4)
EQUAL!
('\n\nThe mind expands far', 5)
CPU times: user 18 s, sys: 1.07 s, total: 19.1 s
Wall time: 24.5 s


'\n\nOpening a door\n\nA new world awaits inside\n\nThe mind expands far'

In [ ]:
haiku_v3("The seasons changing from Fall to Winter.")



Leaves fall away, soon
Snow will blanket the ground

gen poem ['\n\n', 'Leaves', 'fall', 'away', ',', 'soon', '\n', 'Snow', 'will', 'blanket', 'the', 'ground', '\n']
('\n\nLeaves fall away', 4)
{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          81,
          82,
          83,
          85,
          89,
          94,
          99,
          100,
          105,
          106,
          110,
          115,
          123,
          127,
          134
        ],
        "token_logprobs": [
          -0.021073261,
          -3.2616e-05,
          -0.5703003,
          -0.011517642,
          -1.76789,
          -1.6612957,
          -1.1764408,
          -1.1760604,
          -0.22848402,
          -0.6637858,
          -1.1614192,
          -0.7797424,
          -0.056278076,
          -0.5639004,
          -0.23976663
        ],
        "tokens": [
          "\n",
          "\n",
          "Le",
          "ave

In [ ]:
haiku_v3("The taste of a cool lemonade refereshing my friends and I on a very hot day in the U.S.")

('\n\nLemonade so sweet', 5)
('\n\nOn a hot summer day', 6)
EQUAL!
('\n\nOn a hot summer day ', 6)
1
('\n\nWe laugh and smile', 4)
EQUAL!
('\n\nWe laugh and smile in', 5)


'\n\nLemonade so sweet\n\nOn a hot summer day bright\n\nWe laugh and smile in'

In [ ]:
haiku_v3("The car drives incredibly fast.")

('\n\nThe car drives', 3)
EQUAL!
[['\n', '\n\nThe car drives', False], ['\n\n', '\n\nThe car drives', False], ['\n\nThe', '\n\nThe car drives', False], ['\n\nThe car', '\n\nThe car drives', False], ['\n\nThe car drives', '\n\nThe car drives', True]]
('\n\nThe car drives quickly', 5)
('\n\nThe wind whips past my face', 6)
EQUAL!
[['\n', '\n\nThe wind whips past my face', False], ['\n\n', '\n\nThe wind whips past my face', False], ['\n\nThe', '\n\nThe wind whips past my face', False], ['\n\nThe wind', '\n\nThe wind whips past my face', False], ['\n\nThe wind wh', '\n\nThe wind whips past my face', False], ['\n\nThe wind whips', '\n\nThe wind whips past my face', False], ['\n\nThe wind whips past', '\n\nThe wind whips past my face', False], ['\n\nThe wind whips past my', '\n\nThe wind whips past my face', False], ['\n\nThe wind whips past my face', '\n\nThe wind whips past my face', True]]
('\n\nThe wind whips past my face ', 6)
1
('\n\nThe thrill of the speed', 5)


'\n\nThe car drives quickly\n\nThe wind whips past my face free\n\nThe thrill of the speed'

In [ ]:
haiku_v3("The way water flows through the Earth and inside organisms.")



The way water flows
Through the Earth and inside us
Life
gen poem ['\n\n', 'The', 'way', 'water', 'flows', '\n', 'Through', 'the', 'Earth', 'and', 'inside', 'us', '\n', 'Life']
('\n\nThe way water flows', 5)
print line 1 

The way water flows


It nourishes life, a gift from the gods.
gen poem ['\n\n', 'It', 'nourishes', 'life', ',', 'a', 'gift', 'from', 'the', 'gods', '.']
('\n\nIt nourishes life', 5)
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": {
        "text_offset": [
          172,
          173,
          174,
          176,
          181,
          186,
          191,
          192,
          194,
          199,
          204,
          208,
          213,
          214,
          214
        ],
        "token_logprobs": [
          -0.032446254,
          -2.8921473e-05,
          -2.348636,
          -2.3900301,
          -0.0010354641,
          -1.4520656,
          -1.7155892,
          -2.5462115,
          -2.282167,
       

'\n\nThe way water flows\n\nIt nourishes life blessing\n\nA vital part of'